# Dogs vs Cats with Keras

## Loading Liraries

In [0]:
!pip install fastai==0.7.0
!{sys.executable} -m pip install torchtext==0.2.3
!curl http://files.fast.ai/data/dogscats.zip --output sample_data/dogscats.zip
!unzip sample_data/dogscats.zip

In [3]:
!ls

dogscats  sample_data


## Loading Keras Libraries

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
PATH = 'dogscats/'
sz=224
bs=64

In [5]:
!ls {PATH}

models	sample	test1  train  valid


In [0]:
train_data_dir = f'{PATH}train'
valid_data_dir = f'{PATH}valid'

In [0]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                                   rotation_range=10,
                                   vertical_flip=False)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [8]:
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(sz, sz),
                                  batch_size=bs, class_mode='binary')

test_generator = test_datagen.flow_from_directory(valid_data_dir, target_size=(256, 256), 
                                 shuffle=False, batch_size=bs, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: 
  layer.trainable = False

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=3, workers=4,
                    validation_data=test_generator, validation_steps=test_generator.n // bs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
359/359 [==============================] - 335s 935ms/step - loss: 0.2155 - acc: 0.9480 - val_loss: 0.0357 - val_acc: 0.9894
Epoch 2/3
359/359 [==============================] - 319s 888ms/step - loss: 0.0878 - acc: 0.9692 - val_loss: 0.0504 - val_acc: 0.9845
Epoch 3/3
359/359 [==============================] - 317s 883ms/step - loss: 0.0713 - acc: 0.9743 - val_loss: 0.0615 - val_acc: 0.9840
CPU times: user 29min 52s, sys: 53.5 s, total: 30min 45s
Wall time: 16min 11s


In [0]:
split_at = 140
for layer in model.layers[:split_at]:
  layer.trainable = False

for layer in model.layers[split_at:]:
  layer.trainable = True
  
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=1, workers=4,
                    validation_data=test_generator, validation_steps=test_generator.n // bs)

Epoch 1/1
359/359 [==============================] - 354s 986ms/step - loss: 0.0897 - acc: 0.9712 - val_loss: 0.1108 - val_acc: 0.9835
CPU times: user 10min 46s, sys: 29.6 s, total: 11min 15s
Wall time: 5min 55s


In [0]:
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=10, workers=4,
                    validation_data=test_generator, validation_steps=test_generator.n // bs,
                    callbacks=[learning_rate_reduction])

Epoch 1/10
359/359 [==============================] - 351s 978ms/step - loss: 0.0472 - acc: 0.9845 - val_loss: 0.1507 - val_acc: 0.9793
Epoch 2/10
359/359 [==============================] - 324s 903ms/step - loss: 0.0349 - acc: 0.9886 - val_loss: 0.4086 - val_acc: 0.9489
Epoch 3/10
359/359 [==============================] - 321s 894ms/step - loss: 0.0270 - acc: 0.9910 - val_loss: 0.0696 - val_acc: 0.9861
Epoch 4/10
359/359 [==============================] - 322s 897ms/step - loss: 0.0227 - acc: 0.9931 - val_loss: 0.0852 - val_acc: 0.9835
Epoch 5/10
359/359 [==============================] - 319s 889ms/step - loss: 0.0187 - acc: 0.9940 - val_loss: 0.0495 - val_acc: 0.9897
Epoch 6/10
359/359 [==============================] - 318s 885ms/step - loss: 0.0163 - acc: 0.9946 - val_loss: 0.0332 - val_acc: 0.9912
Epoch 7/10
359/359 [==============================] - 313s 871ms/step - loss: 0.0141 - acc: 0.9954 - val_loss: 0.0353 - val_acc: 0.9897
Epoch 8/10
359/359 [============================